# @title
### EP2 MAC0417 / MAC5768
##################################################################
# AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMERO USP,#
# DECLARO QUE SOU O ÚNICO AUTOR E RESPONSÁVEL PELA RESOLUÇÃO #
# DESTE EP. #
# TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM, #
# SEGUINDO AS INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM #
# DESONESTIDADE ACADÊMICA OU PLÁGIO. #
# #
# DECLARO TAMBÉM, QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS #
# DESSE PROGRAMA, E QUE EU NÃO DISTRIBUI OU FACILITEI A #
# SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E #
# DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS #
# DEFINIDOS NO CÓDIGO DE ÉTICA DA USP. #
# #
# ENTENDO QUE JUPYTER NOTEBOOKS SEM ASSINATURA NÃO SERÃO #
# CORRIGIDOS E, AINDA ASSIM, PODERÃO SER PUNIDOS POR #
# DESONESTIDADE ACADÊMICA. #
# #
# #
# Nome : Patrícia da Silva Rodrigues #
# NUSP : 11315590 #
# Turma: MAC0417 #
# Prof.: Ronaldo Fumio Hashimoto #
##################################################################

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread, imsave
import cv2
import csv 


## Gerando metadados do banco de imagens

In [ ]:
if not os.path.exists("Metadados"):
    os.makedirs("Metadados")

funcoes auxiliares

In [ ]:
# Função para separar nomes dos objetos
def separar_nomes(caminho_imagem):
    nome_arquivo = caminho_imagem.split('/')[-1]
    nome_arquivo_sem_extensao = nome_arquivo.split('.')[0]
    palavras = nome_arquivo_sem_extensao.split('_')
    return palavras

In [ ]:
# Função para obter dados da imagem a partir do diretório
def dados_imagem(diretorio):
    partes = diretorio.split('/')
    partes.pop(0)
    partes.pop(-1)
    return partes

gerando metadados do banco original

In [ ]:
def processar_imagens(diretorio_raiz):
    metadados_lista = []
    csv_filename = 'Metadados/metadados.csv'  # Define CSV filename here

    for root, dirs, files in os.walk(diretorio_raiz):
        for file in files:
            if file.endswith(".jpeg") or file.endswith(".jpg"):
                caminho_imagem = os.path.join(root, file)
                dados = dados_imagem(caminho_imagem)  # Assuming dados_imagem is defined elsewhere
                objetos = separar_nomes(caminho_imagem)  # Assuming separar_nomes is defined elsewhere
                
                metadados_lista.append({
                    "nome_imagem": file,
                    # "caminho_imagem": caminho_imagem,
                    "objetos": ",".join(objetos),
                    "fundo": dados[0] if len(dados) > 0 else "",
                    "iluminacao": dados[1] if len(dados) > 1 else "",
                })

    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)

    fieldnames = ['nome_imagem', 'objetos','fundo', 'iluminacao']

    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for metadados in metadados_lista:
            writer.writerow(metadados)

    print(f"Dados escritos no arquivo {csv_filename}!")

In [ ]:
processar_imagens('banco_de_imagens')

# Aplicando a escala em niveis de cinza

funcao para transformar uma imagem em niveis de cinza

In [ ]:
# constantes para a conversao em escala de cinza
RGB_TO_GRAY = [0.2989, 0.5870, 0.1140]

In [ ]:
# Funcao que converte RGB image para grayscale
def rgb2gray(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


Construindo o dataset com as imagens em niveis de cinza

In [ ]:
csv_path = 'Metadados/metadados.csv'
metadata = pd.read_csv(csv_path)



base_dir = 'banco_de_imagens'
new_base_dir = 'originalGrayDataset'


for index, row in metadata.iterrows():
    try:
        image_name = row['nome_imagem']
        image_path = os.path.join(base_dir, row['fundo'], row['iluminacao'], image_name)
        
        if os.path.exists(image_path):
            image = cv2.imread(image_path)
            
            if image is not None:
                image = cv2.resize(image, (0, 0), fx=0.5, fy=0.5)
                gray_image = rgb2gray(image)
                new_image_dir = os.path.join(new_base_dir, row['fundo'], row['iluminacao'])
                os.makedirs(new_image_dir, exist_ok=True)
                new_image_path = os.path.join(new_image_dir, image_name)
                gray_image_uint8 = (gray_image * 255).astype('uint8')  
                cv2.imwrite(new_image_path, gray_image_uint8)
    except KeyError as e:
        print(f"KeyError: {e}")


## Associando as imagens em niveis de cinza ao arquivo de metadados

In [ ]:

# Função para processar as imagens e extrair metadados
def processar_imagens(diretorio_raiz):
    diretorio_metadados = "Metadados_Gray_Image"
    if not os.path.exists(diretorio_metadados):
        os.makedirs(diretorio_metadados)

    metadados_lista = []
    for root, dirs, files in os.walk(diretorio_raiz):
        for file in files:
            if file.endswith(".jpeg") or file.endswith(".jpg"):
                caminho_imagem = os.path.join(root, file)
                dados = dados_imagem(caminho_imagem)
                objetos = separar_nomes(caminho_imagem)

                
                metadados_lista.append({
                    "nome_imagem": file,
                    # "caminho_imagem": caminho_imagem,
                    "objetos": ",".join(objetos),
                    "fundo": dados[0] if len(dados) > 0 else "",
                    "iluminacao": dados[1] if len(dados) > 1 else ""
                })
    # Salva os metadados em um CSV
    metadados_df = pd.DataFrame(metadados_lista)
    if not metadados_df.empty:
        metadados_df.to_csv("Metadados_gray_image/metadados_gray_image.csv", index=False)
    else:
        print("Nenhum metadado foi extraído das imagens.")

    print(f"Dados escritos no arquivo {diretorio_metadados}!")


In [ ]:
# Processar todas as imagens nos subdiretórios
processar_imagens("originalGrayDataset")

# EP2 

## Parte 1 - Data Augmentation (Function Image Tranformation)

funcao que inverte kernel

In [ ]:
def reverte(matriz):
    matriz_revertida = []
    for linha in reversed(matriz):
        matriz_revertida.append(list(reversed(linha)))
    matriz_revertida = np.array(matriz_revertida)
    return matriz_revertida

funcao que aplica o kernel na imagem

In [ ]:
def apply_kernel(image, kernel):
    height, width = image.shape
    k_height, k_width = kernel.shape
    output = np.zeros((height, width))

    pad_height = k_height // 2
    pad_width = k_width // 2

    padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')
    
    output = np.zeros_like(image)
    
    for i in range(height):
        for j in range(width):
            output[i, j] = np.sum(kernel * padded_image[i:i+k_height, j:j+k_width])
    
    return output

contrast stretching 

In [ ]:
def contrast_stretching(image):
    if image is None:
        return None
    
    a, b = np.min(image), np.max(image)
    if a == b:
        return image
    
    stretched_image = (image - a) * (255 / (b - a))
    stretched_image = np.clip(stretched_image, 0, 255).astype(np.uint8)
    
    return stretched_image

log transformation

In [ ]:
def log_transform(image, c):
    return c * np.log1p(image)


exp transformation

In [ ]:
def exp_transform(image, c, gamma):
    return c * np.power(image, gamma)


laplacian filter

In [ ]:
def laplacian_filter(image):
    # kernel Laplaciano
    laplacian_kernel = np.array([[0, 1, 0],
                                 [1, -4, 1],
                                 [0, 1, 0]])
    
    # Aplicando a correlação na imagem usando o kernel Laplaciano
    laplacian_image = apply_kernel(image, laplacian_kernel)
    
    return laplacian_image

mean filter

In [ ]:
def mean_filter(image):
    mean_kernel = np.ones((3, 3)) / 9
    mean_kernel = reverte( mean_kernel) #invertendo o kernel para a convolucao
    mean_image = apply_kernel(image, mean_kernel) # aplicando

    return mean_image

aplicando a transformacao 

In [ ]:
transformations = {
    'contrast_stretching': contrast_stretching,
    'log_transform': lambda image, c=2: log_transform(image, c),  # aqui podemos definir o valor das constantes 
    'exp_transform': lambda image, c=2, gamma=3: exp_transform(image, c, gamma), # aqui podemos definir o valor das constantes 
    'laplacian_filter': laplacian_filter,
    'mean_filter': mean_filter,
}


metadados_augmentedDataset_image = []
base_dir = 'originalGrayDataset'
csv_path = 'Metadados_gray_image/metadados_gray_image.csv'
new_base_dir = 'augmentedDataset'

metadata = pd.read_csv(csv_path)

if not os.path.exists(new_base_dir):
    os.makedirs(new_base_dir)

for index, row in metadata.iterrows():
    caminho_imagem = os.path.join(base_dir, row['fundo'], row['iluminacao'], row['nome_imagem'])
    grayscale_image = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)

    if grayscale_image is None:
        print(f"Erro: Não foi possível ler a imagem {caminho_imagem}")
        continue

    for transformation_name, transformation_func in transformations.items():
        output_dir = os.path.join(new_base_dir, transformation_name, row['fundo'], row['iluminacao'])

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        transformed_image = transformation_func(grayscale_image)

        if transformed_image is None:
            print(f"Erro na transformação {transformation_name} para a imagem {caminho_imagem}")
            continue
            
        output_image_path = os.path.join(output_dir, row['nome_imagem'])
        cv2.imwrite(output_image_path, transformed_image)

        # Registro dos metadados no DataFrame
        metadados_augmentedDataset_image.append({
            'nome_imagem': row['nome_imagem'],
            'iluminacao': row['iluminacao'],
            'fundo': row['fundo'],
            'filtro': transformation_name
        })



gera os metadados da nova base de dados (augmentedDataset)

In [ ]:
import csv

csv_filename = 'Metadados_Augmented_Data/metadados_augmented.csv'
os.makedirs(os.path.dirname(csv_filename), exist_ok=True)

fieldnames = ['nome_imagem', 'fundo', 'iluminacao', 'filtro']

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for metadados in metadados_augmentedDataset_image:
        writer.writerow(metadados)

print(f"Dados escritos no arquivo {csv_filename}!")
